## 2. Forecasting & Modeling Notebook

#### 1. Imports and Data Loading

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import joblib
import os

# Load data
df = pd.read_csv('sales_data.csv')

# Fix column typo
df.rename(columns={'SerailNum': 'SerialNum'}, inplace=True)

# Parse dates robustly (handles all formats in your file)
df['weekend_date'] = pd.to_datetime(df['weekend_date'], dayfirst=True, errors='coerce')

# Drop rows with invalid or missing dates/quantities
df = df.dropna(subset=['weekend_date', 'quantity'])
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df = df.dropna(subset=['quantity'])
df['SerialNum'] = df['SerialNum'].astype(str)

#### 2. Filling Missing Weeks & Aggregating Duplicates

In [71]:
def fill_missing_weeks(data):
    # Aggregate duplicates (same SerialNum, same weekend_date)
    data_agg = data.groupby(['weekend_date', 'SerialNum', 'channel', 'brand', 'category', 'sub_category'], as_index=False)['quantity'].sum()
    
    # Create a full weekly date range for this product
    all_weeks = pd.date_range(data_agg['weekend_date'].min(), data_agg['weekend_date'].max(), freq='W-SAT')
    base = {
        'SerialNum': data_agg['SerialNum'].iloc[0],
        'channel': data_agg['channel'].iloc[0],
        'brand': data_agg['brand'].iloc[0],
        'category': data_agg['category'].iloc[0],
        'sub_category': data_agg['sub_category'].iloc[0]
    }
    full = pd.DataFrame({'weekend_date': all_weeks})
    for col in ['SerialNum', 'channel', 'brand', 'category', 'sub_category']:
        full[col] = base[col]
    merged = pd.merge(full, data_agg[['weekend_date', 'quantity']], on='weekend_date', how='left')
    merged['quantity'] = merged['quantity'].fillna(0)
    return merged

# Apply to every SerialNum
df_filled = pd.concat([fill_missing_weeks(g) for _, g in df.groupby('SerialNum')], ignore_index=True)

#### 3. Train/Validation Split

In [72]:
# Validation period: Jun-Jul-Aug 2024
val_start = pd.Timestamp('2024-06-01')
val_end = pd.Timestamp('2024-08-31')

# Forecast period: Sept-Oct-Nov 2024
forecast_start = pd.Timestamp('2024-09-01')
forecast_end = pd.Timestamp('2024-11-30')

def split_data(data):
    train = data[data['weekend_date'] < val_start].copy()
    valid = data[(data['weekend_date'] >= val_start) & (data['weekend_date'] <= val_end)].copy()
    return train, valid

#### 4. Modeling, Validation and Forecasting

In [73]:
results = []
accuracy_report = []

for serial in df_filled['SerialNum'].unique():
    data = df_filled[df_filled['SerialNum'] == serial].copy()
    train, valid = split_data(data)
    prophet_train = train[['weekend_date', 'quantity']].rename(columns={'weekend_date': 'ds', 'quantity': 'y'})
    
    # Prophet model
    model = Prophet(weekly_seasonality=True, yearly_seasonality=True, daily_seasonality=False)
    model.fit(prophet_train)
    
    # Save model
    os.makedirs('models', exist_ok=True)
    joblib.dump(model, f'models/prophet_serial_{serial}.pkl')
    
    # Validation forecast
    periods_val = valid['weekend_date'].nunique()
    future_val = model.make_future_dataframe(periods=periods_val, freq='W-SAT')
    future_val = future_val[future_val['ds'].between(val_start, val_end)]
    forecast_val = model.predict(future_val)
    val_results = valid[['weekend_date', 'quantity']].merge(
        forecast_val[['ds', 'yhat']], left_on='weekend_date', right_on='ds', how='left'
    )
    val_results['month'] = val_results['weekend_date'].dt.to_period('M')
    monthly_accuracy = (
        val_results.groupby('month')
        .apply(lambda x: 1 - np.sum(np.abs(x['yhat'] - x['quantity'])) / np.sum(x['quantity']) if np.sum(x['quantity']) > 0 else np.nan)
        .reset_index().rename(columns={0: 'accuracy'})
    )
    monthly_accuracy['SerialNum'] = serial
    accuracy_report.append(monthly_accuracy)
    
    # Forecast for Sept-Nov 2024
    last_date = data['weekend_date'].max()
    weeks_to_forecast = ((forecast_end - last_date).days // 7)
    future = model.make_future_dataframe(periods=weeks_to_forecast, freq='W-SAT')
    future_forecast = model.predict(future)
    forecast_needed = future_forecast[future_forecast['ds'].between(forecast_start, forecast_end)]
    forecast_needed['SerialNum'] = serial
    results.append(forecast_needed[['ds', 'SerialNum', 'yhat']].rename(columns={'ds': 'weekend_date', 'yhat': 'forecast_quantity'}))

# Combine all forecasts and accuracy reports
forecast_df = pd.concat(results, ignore_index=True)
accuracy_df = pd.concat(accuracy_report, ignore_index=True)

01:18:59 - cmdstanpy - INFO - Chain [1] start processing
01:19:00 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\KIIT\AppData\Local\Temp\ipykernel_10224\2942445104.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: 1 - np.sum(np.abs(x['yhat'] - x['quantity'])) / np.sum(x['quantity']) if np.sum(x['quantity']) > 0 else np.nan)
01:19:00 - cmdstanpy - INFO - Chain [1] start processing
01:19:01 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\KIIT\AppData\Local\Temp\ipykernel_10224\2942445104.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be e

#### 5. Save and Present Results

In [74]:
# Save outputs
forecast_df.to_csv('forecast_sept_nov_2024.csv', index=False)
accuracy_df.to_csv('validation_accuracy_jun_aug_2024.csv', index=False)

# Show sample forecast
print("Sample Forecast for Sept–Nov 2024:")
print(forecast_df.head())

# Show accuracy table
print("Monthly Validation Accuracy (Jun–Aug 2024):")
print(accuracy_df)

Sample Forecast for Sept–Nov 2024:
  weekend_date SerialNum  forecast_quantity
0   2024-09-07         5         451.228212
1   2024-09-14         5         637.747514
2   2024-09-21         5         596.884262
3   2024-09-28         5         333.765216
4   2024-10-05         5          59.603762
Monthly Validation Accuracy (Jun–Aug 2024):
     month   accuracy SerialNum
0  2024-06   0.649995         1
1  2024-07   0.332449         1
2  2024-08   0.243575         1
3  2024-06   0.407209         2
4  2024-07   0.502371         2
5  2024-08   0.127261         2
6  2024-06   0.314458         4
7  2024-07   0.293139         4
8  2024-08   0.483990         4
9  2024-06 -59.094687         5


#### Summary

In [76]:
print("Forecasts for September–November 2024 generated for each SerialNum.")
print("Validation accuracy for June–August 2024 is reported per month and SerialNum.")
print("Models are saved in the 'models/' directory for reproducibility.")

Forecasts for September–November 2024 generated for each SerialNum.
Validation accuracy for June–August 2024 is reported per month and SerialNum.
Models are saved in the 'models/' directory for reproducibility.
